In [4]:
import cv2
import numpy as np
import pandas as pd
import os

df = pd.read_csv('validated.csv')

df = df.dropna()

df = df[df['valid'] == True]

old_df = None
if os.path.exists('classified.csv'):
    old_df = pd.read_csv('classified.csv')
    
# remove if already classified and image and mask match
if old_df is not None:
    df = df[~df['mask'].isin(old_df['mask'])]

df

,image,x,y,mask,valid


In [2]:
cv2.namedWindow('image')
cv2.resizeWindow('image', 800, 640)

states = ['unknown', 'unripe', 'ripening', 'ripe']
selected_class = states[0]
goto_next = False

def get_on_press(class_name):
    def on_press(state, *args, **kwargs):
        global selected_class, goto_next
        selected_class = class_name
        goto_next = True
        return state
    return on_press

for state in states:
    cv2.createButton(state, get_on_press(state), None, cv2.QT_PUSH_BUTTON, 0)


def get_display_image(row):
    cv2image = cv2.imread(row.image)

    mask = np.load(row['mask'])
    mask = mask.astype(np.uint8)
    
    # apply the mask to the image
    mask_image = cv2.bitwise_and(cv2image, cv2image, mask=mask)
    
    # crop the image to the mask
    mask_image = mask_image[np.ix_(mask.any(1),mask.any(0))]
    
    # double the size of the image
    mask_image = cv2.resize(mask_image, (mask_image.shape[1] * 2, mask_image.shape[0] * 2))
    
    return mask_image


image_index = 0
data = []

show_image = get_display_image(df.iloc[image_index])

while True:
    if show_image is None or show_image.shape[0] <= 0 or show_image.shape[1] <= 0:
        print(f'invalid mask {df.iloc[image_index]["mask"]}')
        image_index += 1
        show_image = get_display_image(df.iloc[image_index])
        continue
        
    cv2.imshow("image", show_image)
    k = cv2.waitKey(50) & 0xFF
    
    # if escape key, exit
    if k == 27:
        break
    
    if goto_next or k == ord('n'):
        goto_next = False
        row = df.iloc[image_index]
        data.append([row.image, row.x, row.y, row['mask'], selected_class])
        image_index += 1
        if image_index >= len(df):
            break

        show_image = get_display_image(df.iloc[image_index])

# close all open windows
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jack/.local/share/virtualenvs/datadev-5ox7fytP/lib/python3.10/site-packages/cv2/qt/plugins"


In [3]:
df = pd.DataFrame(data, columns=['image', 'x', 'y', 'mask', 'lifecycle'])

if old_df is not None:
    df = pd.concat([old_df, df])

df.to_csv('classified.csv', index=False)

df

,image,x,y,mask,lifecycle
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripening
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,324,300,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unknown
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,272,321,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unknown
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,364,288,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unknown
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,109,178,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripe
...,...,...,...,...,...
286,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,192,160,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripening
287,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,331,55,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
288,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,368,18,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
289,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,331,102,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe


In [4]:
df['lifecycle'].value_counts()

unripe      1371
ripe         532
ripening     384
unknown      109
overripe       2
Name: lifecycle, dtype: int64